In [1]:
#!pip install meshio
#!pip install gcsfs

  Using cached meshio-4.3.6-py3-none-any.whl (150 kB)


In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import xarray as xr
import meshio
import gcsfs

In [3]:
def vtu_transform()
    projname = 'ldeo-glaciology'
    bucket = 'ldeo-glaciology/elmer_janie/output_vals/'

    fs = gcsfs.GCSFileSystem(project=projname)
    filelist = fs.ls(bucket)


    #info about mesh grid (set by user in model - mesh.grd file)
    xgrid_cells = 1000
    ygrid_cells = 30

    xgrid_pts = xgrid_cells + 1
    ygrid_pts = ygrid_cells + 1

    xnodes = np.array(range(xgrid_pts)) #ynodes 
    ynodes = np.array(range(ygrid_pts))  # xnodes
    tsteps = np.array(range(len(filelist)))  #number of time steps

    timestep = [] #list of data arrays
    for path in filelist:
        t_val = filelist.index(path)
        print(path)
        #print(t_val)
        gcs_file = fs.get(path,'out.vtu') #load file
        mesh = meshio.read('out.vtu', file_format= 'vtu') #mesh the file

        velx = mesh.point_data['velocity'][:,0].reshape((ygrid_pts,xgrid_pts)) #reshape the mesh coordinates
        vely = mesh.point_data['velocity'][:,1].reshape((ygrid_pts,xgrid_pts))

        xdim = mesh.points[:,0].reshape((ygrid_pts,xgrid_pts))
        ydim = mesh.points[:,1].reshape((ygrid_pts,xgrid_pts))

        tempdataset = xr.Dataset(data_vars = {'vel_x' : (('ynode','xnode'), velx) ,'vel_y' :(('ynode','xnode'), vely)},
                          coords = {'yvals': (('ynode','xnode'),ydim),'xvals': (('ynode','xnode'),xdim)})
        timestep.append(tempdataset)

    datacomb = xr.concat(timestep, dim='t')
    return datacomb

ldeo-glaciology/elmer_janie/output_vals/Output0001.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0002.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0003.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0004.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0005.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0006.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0007.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0008.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0009.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0010.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0011.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0012.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0013.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0014.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0015.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0016.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0017.vtu
ldeo-glaciology/elmer_janie/output_vals/Output0018.vtu
ldeo-glaci

In [ ]:
def surface_deriv(datacomb):
    surface = datacomb.yvals.sel(ynode = (datacomb.ynode.shape[0] - 1))

    dx = (surface.xvals[0,1] - surface.xvals[0,0]).item()
    dysurf = np.zeros(surface.shape)
    dy2surf = np.zeros(surface.shape)
    #calculate derivatives of profiles in each time step 
    for w in range(len(surface.t)):
        dysurf[w,1:] = (surface.sel(t = w)[1:] - surface.sel(t = w)[0:-1])/dx
        dy2surf[w,1:-1] = (surface.sel(t = w)[0:-2] - 2*surface.sel(t = w)[1:-1] + surface.sel(t = w)[2:])/(dx**2)

    #create dataset from derivatives
    der = xr.Dataset(data_vars = {'dy' : (('t','xnode'), dysurf), 
                                 'dy2' : (('t','xnode'), dy2surf)})

    #merge derivatives dataset with surface dataset 
    surfder = xr.merge([surface.drop('yvals'),der])
    return surfder

In [ ]:
def surf_analyze(dataset, start_idx):
    T = dataset.t.shape[0]
    
    peakarr = []
    inflarr = []
    concarr = [] 
    for k in range(start_idx,dataset.t.shape[0]):
        print(k)
        singleslice = dataset.sel(t = k)
        peakind = 0
        inflind = []
        L = singleslice.dy2.shape[0]

        for i in range(L-1):
            if singleslice.dy[i]*singleslice.dy[i+1] < 0:
                peakind = i
            if singleslice.dy2[i]*singleslice.dy2[i+1] < 0:
                inflind.append(i)
        c_l = singleslice.dy2[0:peakind].argmax()
        c_r = singleslice.dy2[peakind:].argmax() + peakind


        peakarr.append(datacomb.sel(t = k, xnode = peakind))

        #variable number of inflection points (max 4)
        inflset = datacomb.sel(t = k, xnode = inflind)
        l = 4 - len(inflind)
        print(l)
        if l > 0:
            fill = xr.full_like(inflset.sel(xnode = slice(0,l)),np.nan)
            inflsetfil = xr.concat([inflset,fill],dim = 'xnode')
            inflarr.append(inflsetfil)
        else:
            inflarr.append(inflset)
            
    peakdat = xr.concat(peakarr, dim='t')
    concdat = xr.concat(concarr, dim='t')
    infldat = xr.concat(inflarr,dim='t')
    return peakdat,concdat,infldat